## MNIST + Convolutional Neural Network

In [1]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Conv2D, MaxPooling2D
from keras.datasets import mnist
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
img_rows = img_cols = 28
num_classes = 10

(x_train, y_train),(x_test, y_test) = mnist.load_data()

# Reshape from (60000, 28,28) to (60000,28,28,1) : 60000 samples, 28x28 pixels, 1 channel (B/W)
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
# Reshape from (10000, 28,28) to (10000,28,28,1) : 10000 samples, 28x28 pixels, 1 channel (B/W)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)

In [3]:
print (x_train.shape)
print (y_train.shape)

(60000, 28, 28, 1)
(60000,)


In [4]:
# Normalize pixel values: [0-255] --> [0.0-1.0]
x_train, x_test = x_train / 255.0, x_test / 255.0

# One-hot encode labels
# We could skip this step and use the 'sparse_categorical_crossentropy' loss function instead
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [5]:
print (y_train[:1])

[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]]


In [6]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(28,28,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy'])

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1568)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               200832    
_________________________________________________________________
dense_2 (Dense)              (None, 10)               

In [7]:
batch_size = 128
epochs = 10

In [8]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=batch_size, epochs=epochs)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 23s 382us/step - loss: 0.2183 - accuracy: 0.9358 - val_loss: 0.0604 - val_accuracy: 0.9813
Epoch 2/10
60000/60000 [==============================] - 23s 377us/step - loss: 0.0594 - accuracy: 0.9822 - val_loss: 0.0489 - val_accuracy: 0.9828
Epoch 3/10
60000/60000 [==============================] - 23s 376us/step - loss: 0.0404 - accuracy: 0.9877 - val_loss: 0.0508 - val_accuracy: 0.9831
Epoch 4/10
60000/60000 [==============================] - 23s 378us/step - loss: 0.0307 - accuracy: 0.9905 - val_loss: 0.0353 - val_accuracy: 0.9876
Epoch 5/10
60000/60000 [==============================] - 23s 379us/step - loss: 0.0241 - accuracy: 0.9926 - val_loss: 0.0355 - val_accuracy: 0.9889
Epoch 6/10
60000/60000 [==============================] - 23s 377us/step - loss: 0.0198 - accuracy: 0.9936 - val_loss: 0.0259 - val_accuracy: 0.9909
Epoch 7/10
60000/60000 [==============================] 

In [9]:
score = model.evaluate(x_test, y_test)
print(score)

10000/10000 [==============================] - 1s 145us/step
[0.03458589150244152, 0.9890999794006348]


In [10]:
model.save("cnn-mnist")

## Predict home-made digits

In [25]:
import numpy as np
import cv2

np.set_printoptions(precision=2, suppress=True)

def loadImage(filename):
        img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
        #img = imread(filename, mode='L')
        img = img / 255
        # Reshape from (28,28) to (1,28,28,1) : 1 sample, 28x28 pixels, 1 channel (B/W)
        img = np.expand_dims(img, axis=0)
        img = np.expand_dims(img, axis=0)
        img = np.reshape(img, (1,img_cols,img_rows,1))
        return np.array(img)

In [26]:
from keras.models import load_model
model = load_model("cnn-mnist")
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1568)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               200832    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total para

In [31]:
img_rows = img_cols = 28
img = "9.png"
img = loadImage(img)
classes = model.predict(img)
predicted = np.argmax(classes)
result = ''
print(" *%s* %s %s" % (digit, classes, result))

 *9* [[0.   0.   0.11 0.   0.01 0.   0.   0.   0.   0.88]] 
